# VIEWS

- We often have to create a complex query and use it multiple times
- Example
    - Create a Book Summary to include
        - Title
        - Author
        - Price
        - Cover
        - Votes
        - Rating
    -

In [3]:
SELECT 
    TITLE,
    AUTHOR,
    PRICE,
    COVER,
    COUNT(RATING) VOTES,
    ROUND(AVG(CAST(RATING AS FLOAT)),2) AS RATING
FROM
    BOOKS
LEFT JOIN
    REVIEWS
ON
    BOOKS.ID=BOOK_ID
GROUP BY
    TITLE,
    AUTHOR,
    PRICE,
    COVER


Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:00.007

TITLE,AUTHOR,PRICE,COVER,VOTES,RATING
Manas,Ramdhari Singh Dinkar,100.00,manas-2.png,0,NULL
Manas,Vivek Dutta Mishra,199.00,manas.png,4,4.5
Summons,John Grishma,400.00,summons.png,0,NULL
The Accursed God,Vivek Dutta Mishra,299.00,tag.png,5,4.6
The Count of Monte Cristo,Alexandre Dumas,499.00,cristo.png,3,4.33


## CHALLENGES

- We may need this information at multiple places forcing us to rewrite the query
- We may need to further customise this query 
    - sort on title
    - sort of rating
    - select only the title and rating
    - filter all books with ratings\>4

### Example: Filter all books that have at least 3 votes

In [14]:
SELECT 
    TITLE,
    AUTHOR,
    PRICE,
    COVER,
    COUNT(RATING) VOTES,
    ROUND(AVG(CAST(RATING AS FLOAT)),2) AS RATING
FROM
    BOOKS

LEFT JOIN
    REVIEWS
ON
    BOOKS.ID=BOOK_ID

GROUP BY
    TITLE,
    AUTHOR,
    PRICE,
    COVER
HAVING
    COUNT(RATING)>0
ORDER BY
    AUTHOR

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.010

TITLE,AUTHOR,PRICE,COVER,VOTES,RATING
The Count of Monte Cristo,Alexandre Dumas,499.00,cristo.png,3,4.33
Manas,Vivek Dutta Mishra,199.00,manas.png,4,4.5
The Accursed God,Vivek Dutta Mishra,299.00,tag.png,5,4.6


# View

- A View can be considered as a 
    
    - VIRTUAL TABLE
        
        - acts and works like a table 
        - but it is actually the result of a query
        - View works on data from other table
            - It doesn't have its own data
    - A NAMED QUERY
        
        - We specify a name for a query
        - we can access the information using the name
        
- A View unlike a regular query is peristed in the datbase
    
    - when you write  a query it is fire and forget
    - we need to write the query again if we need to use it again
    - a view is saved in the database (just like table)
- A View is readonly
    
    - It can only use SELECT query
    - It can't update any stored value
- A View is dynamic
    
    - when underlying table changes, view returns updated result

## CREATE A VIEW

In [18]:
DROP VIEW IF EXISTS BOOKS_SUMMARY
GO

CREATE VIEW BOOKS_SUMMARY
AS
SELECT 
    TITLE,
    AUTHOR,
    PRICE,
    COVER,
    COUNT(RATING) VOTES,
    ROUND(AVG(CAST(RATING AS FLOAT)),2) AS RATING
FROM
    BOOKS
LEFT JOIN
    REVIEWS
ON
    BOOKS.ID=BOOK_ID
GROUP BY
    TITLE,
    AUTHOR,
    PRICE,
    COVER


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

### SIDE NOTE ABOUT "GO"

  

- It is NOT  a SqlServer command
    - Sql Server doesn't understand it
- It is a directive for management toolks like
    - SSMS
    - Azure studio
- It sends all the command before GO in one batch
- remainging command following GO will be in the next batch

  

## Using View

- We can use the view in the same way as we use table (or nested query)
- We can
    - select
    - join
    - filter
    - order
-

In [19]:
SELECT * FROM BOOKS_SUMMARY

Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:00.018

TITLE,AUTHOR,PRICE,COVER,VOTES,RATING
Manas,Ramdhari Singh Dinkar,100.00,manas-2.png,0,NULL
Manas,Vivek Dutta Mishra,199.00,manas.png,4,4.5
Summons,John Grishma,400.00,summons.png,0,NULL
The Accursed God,Vivek Dutta Mishra,299.00,tag.png,5,4.6
The Count of Monte Cristo,Alexandre Dumas,499.00,cristo.png,3,4.33


In [21]:
SELECT 
    TITLE,
    AUTHOR,
    PRICE,
    VOTES,
    RATING
FROM
    BOOKS_SUMMARY
WHERE
    VOTES>0
ORDER BY
    RATING DESC

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.011

TITLE,AUTHOR,PRICE,VOTES,RATING
The Accursed God,Vivek Dutta Mishra,299.00,5,4.6
Manas,Vivek Dutta Mishra,199.00,4,4.5
The Count of Monte Cristo,Alexandre Dumas,499.00,3,4.33


### View is alsways updated if underlying details changes

In [22]:
INSERT INTO BOOKS(TITLE, AUTHOR, PRICE, COVER)
    VALUES('Rashmirathi','Ramdhari Singh Dinkar',99, 'rashmirathi.png');

DECLARE @ID INT -- CREATE A LOCAL VARIABLE ID

SELECT  @ID=MAX(ID) FROM BOOKS -- ASSING THE HIGHEST ID IN BOOKS TO ID VARIABLE

INSERT INTO REVIEWS(BOOK_ID, REVIEWER, RATING)
            VALUES 
                    (@ID, 'Vivek', 5),
                    (@ID, 'Sanjay',4),
                    (@ID, 'Shivanshi',4);


(1 row affected)

(3 rows affected)

Total execution time: 00:00:00.020

### Now our View will include the latest data

In [25]:
SELECT  TITLE, RATING, VOTES FROM BOOKS_SUMMARY WHERE VOTES>0

Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:00.010

TITLE,RATING,VOTES
Manas,4.5,4
Rashmirathi,4.33,3
The Accursed God,4.6,5
The Count of Monte Cristo,4.33,3


In [26]:
SELECT
    VOTES,
    COUNT(*) AS BOOKS
FROM
    BOOKS_SUMMARY
GROUP BY
    VOTES


Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:00.009

VOTES,BOOKS
0,2
3,2
4,1
5,1


## ALTER VIEWS

- Alter view can allow us to change the view query
- We need to specify the entire View query
- It would be almost similar to dropping view and re-creating

In [27]:
ALTER VIEW BOOKS_SUMMARY
AS
SELECT 
    TITLE,
    AUTHOR AS AUTHOR_NAME,
    PRICE,
    COVER,
    COUNT(RATING) VOTES,
    ROUND(AVG(CAST(RATING AS FLOAT)),2) AS RATING
FROM
    BOOKS
LEFT JOIN
    REVIEWS
ON
    BOOKS.ID=BOOK_ID
GROUP BY
    TITLE,
    AUTHOR,
    PRICE,
    COVER


Commands completed successfully.

Total execution time: 00:00:00.010

In [28]:
SELECT DISTINCT(AUTHOR_NAME) FROM BOOKS_SUMMARY

(4 rows affected)

Total execution time: 00:00:00.009

AUTHOR_NAME
Alexandre Dumas
John Grishma
Ramdhari Singh Dinkar
Vivek Dutta Mishra


### CREATE A AUTHORS VIEW

IT SHOULD INCLUDE

- AUTHOR NAME
- BOOKS WRITTEN
- RATING
- AVG BOOK PRICE

In [2]:
CREATE VIEW AUTHORS_VIEW
AS
SELECT
    AUTHOR AS NAME,
    COUNT(TITLE) AS BOOKS_WRITTEN,
    COUNT(RATING) AS VOTES,
    FORMAT(AVG(CAST(RATING AS FLOAT)),'N2') AS RATING,
    FORMAT(AVG(CAST(PRICE AS FLOAT)),'N2') AS AVG_PRICE
FROM
    BOOKS
LEFT JOIN
    REVIEWS
ON
    BOOKS.ID=BOOK_ID
GROUP BY
    AUTHOR

Commands completed successfully.

Total execution time: 00:00:00.022

In [3]:
SELECT * FROM AUTHORS_VIEW

Warning: Null value is eliminated by an aggregate or other SET operation.

(4 rows affected)

Total execution time: 00:00:00.026

NAME,BOOKS_WRITTEN,VOTES,RATING,AVG_PRICE
Alexandre Dumas,3,3,4.33,499.00
John Grishma,1,0,NULL,400.00
Ramdhari Singh Dinkar,4,3,4.33,99.25
Vivek Dutta Mishra,9,9,4.56,254.56


## CREATE REVIEWER VIEW TO INCLUDE

- REVIEWER NAME
- TOTAL REVIEWS GIVEN
- AVERAGE RATING

# STORED PROCEDURE

- STORED PROCEDURES ARE LIKE FUNCTIONS IN PROGRAMMING LANGUAGES
- AS THE NAME SUGGESTS, THEY ARE STORED INSIDE DATABASE AND CAN BE REUSED.
- THEY CAN EXECUTE SOME TASK
- UNLIKE VIEW, A STORED PROCEDURE CAN PERFORM ENTIRE SET OF CRUD OPERATIONS
- IT IS NOT VIEWED AS A TABLE (UNLIKE VIEW)
- IT CAN'T BE USED DIRECTLY INSIDE A SQL QUERY
- IT NEEDS TO BE EXECUTED USING EXEC COMMAND

### LET'S CREATE A SIMPLE STORED PROCEDURE TO RETURN A REVIEWER SUMMARY

In [4]:
CREATE PROCEDURE SP_REVIEWERS
AS
BEGIN
SELECT
    REVIEWER AS NAME,
    COUNT(RATING) AS BOOKS_REVIEWED,
    ROUND(AVG(CAST(RATING AS FLOAT)),2) AS RATING
FROM
    BOOKS
JOIN
    REVIEWS
ON
    BOOKS.ID=BOOK_ID
GROUP BY
    REVIEWER

END

Commands completed successfully.

Total execution time: 00:00:00.015

INVOKING STORED PROCEDURE

- It is different from regular query

In [5]:
EXEC SP_REVIEWERS

(7 rows affected)

Total execution time: 00:00:00.018

NAME,BOOKS_REVIEWED,RATING
Amit,1,5
Prabhat,2,5
Rajesh,1,5
Reena,2,4
Sanjay,3,4.33
Shivanshi,4,4
Vivek,2,5


## SP WITH PARAMETERS

- SP Can take parameters

  

### SP TO FIND ALL BOOKS BY A GIVEN AUTHOR

In [6]:
CREATE PROCEDURE sp_books_by_author
@author_name VARCHAR(100)

AS
BEGIN
    SELECT
        * 
    FROM 
        BOOKS
    WHERE
        AUTHOR=@author_name
END

Commands completed successfully.

Total execution time: 00:00:00.016

### Executing Stored Procedure and passing positional parameter

In [7]:
EXEC SP_BOOKS_BY_AUTHOR 'Vivek Dutta Mishra'

(2 rows affected)

Total execution time: 00:00:00.019

TITLE,AUTHOR,PRICE,COVER,ID
Manas,Vivek Dutta Mishra,199.00,manas.png,2
The Accursed God,Vivek Dutta Mishra,299.00,tag.png,5


## Executing SP and passing named parameter

- Useful for passing multiple parameters
- we can pass in a order different from how it is specified.

In [9]:
EXEC SP_BOOKS_BY_AUTHOR @author_name='Alexandre Dumas'

(1 row affected)

Total execution time: 00:00:00.007

TITLE,AUTHOR,PRICE,COVER,ID
The Count of Monte Cristo,Alexandre Dumas,499.00,cristo.png,6


## SP CAN PERFORM FULL CRUD

- Unlike Views SP can also insert/update/delete
- They can also have conditional values

### SP to insert a review

- We want to insert the record and return the id of the inserted record.

In [10]:
CREATE PROCEDURE SP_ADD_REVIEW
    @BOOK_ID INT, --ID OF THE BOOK
    @REVIEWER VARCHAR(100),
    @RATING INT 
AS
BEGIN
    INSERT INTO REVIEWS(BOOK_ID, REVIEWER, RATING)
    VALUES(@BOOK_ID,@REVIEWER,@RATING)
    return SCOPE_IDENTITY() --THE GENERATED ID FOR THE RECORD
END

Commands completed successfully.

Total execution time: 00:00:00.013

In [17]:
EXEC SP_ADD_REVIEW @BOOK_ID=5, @REVIEWER='Ayaz Khan', @RATING=4

(1 row affected)

Total execution time: 00:00:00.016

In [19]:
EXEC SP_ADD_REVIEW 2, 'Ayaz Khan', 5

(1 row affected)

Total execution time: 00:00:00.017

### SHOULD FAIL FOR INVALID BOOKS

In [20]:
EXEC SP_ADD_REVIEW 10, 'Ayaz Khan', 5

: Msg 547, Level 16, State 0, Procedure SP_ADD_REVIEW, Line 7
The INSERT statement conflicted with the FOREIGN KEY constraint "FK__BOOK_ID__BOOKS__ID". The conflict occurred in database "books_db_g7cr_202407", table "dbo.BOOKS", column 'ID'.

The statement has been terminated.

The 'SP_ADD_REVIEW' procedure attempted to return a status of NULL, which is not allowed. A status of 0 will be returned instead.

Total execution time: 00:00:00.010

HOW TO CAPTURE THE RETURN VALUE OF A SP

In [22]:
DECLARE @new_id INT;

EXEC @new_id = SP_ADD_REVIEW @RATING=4, @BOOK_ID=2, @REVIEWER='Ayaz Khan'

SELECT @new_id as new_id

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.005

new_id
29


In [23]:
SELECT 
    TITLE,
    REVIEWER,
    RATING 
FROM 
    REVIEWS
JOIN
    BOOKS
ON
    BOOKS.ID=BOOK_ID
WHERE
    REVIEWS.ID=29


(1 row affected)

Total execution time: 00:00:00.005

TITLE,REVIEWER,RATING
Manas,Ayaz Khan,4


In [24]:
SELECT @new_id as new_id

: Msg 137, Level 15, State 2, Line 1
Must declare the scalar variable "@new_id".

Total execution time: 00:00:00

## CREATE A SP TO ADD REVIEW BY PARTIAL BOOK TITLE

- We will pass
    - partial book title
    - reviewer
    - rating
- SP should check if search matches a single book
    - if yes
        - insert the review for that record
    - if no
        - send a error message indicating the problem

- return success or failure
- also return the last id generated on success

In [45]:
DROP PROCEDURE IF EXISTS SP_ADD_REVIEW_BY_TITLE;

Commands completed successfully.

Total execution time: 00:00:00.015

In [46]:
CREATE PROCEDURE SP_ADD_REVIEW_BY_TITLE
    @TITLE VARCHAR(100),
    @REVIEWER VARCHAR(100),
    @RATING INT
    --@REVIEW_ID INT OUTPUT, --THIS WILL BE RETURNED BY SP
    --@BOOK_ID_REVIEWED INT OUTPUT
AS
BEGIN
    -- STEP 0. WE NEED TO STORE COUNT AND ID
    DECLARE @COUNT INT;
    DECLARE @BOOK_ID INT;

    -- STEP 1. FIND HOW MANY BOOKS MATCHES THE TITLE
    SELECT
        @COUNT=COUNT(*), --TOTAL MATCHING BOOKS STORE IN COUNT
        @BOOK_ID=MAX(ID)   --ID OF MATCHING BOOK. STORE IN BOOK_ID
    FROM
        BOOKS
    WHERE
        TITLE LIKE '%'+@TITLE+'%';


    -- STEP 2. RAISE ERROR IF BOOK COUNT != 1
    IF @COUNT=0
    BEGIN
        RAISERROR('No Matching Book',15,1)
        RETURN 0;
    END

    IF @COUNT>1
    BEGIN
        RAISERROR('Multiple Matching Books',15,2)
        RETURN 0;
    END

    -- STEP 3. INSERT RECORD IN CASE THERE IS ONLY ONE MATCHING BOOK
    


    DECLARE @REVIEW_ID INT;
    EXEC @REVIEW_ID = SP_ADD_REVIEW @BOOK_ID, @REVIEWER, @RATING

    RETURN @REVIEW_ID;

    --@BOOK_ID_REVIEWED=@BOOK_ID;

END


Commands completed successfully.

Total execution time: 00:00:00.048

In [48]:

EXEC SP_ADD_REVIEW_BY_TITLE @TITLE='God', @REVIEWER='Santosh', @Rating=4

(1 row affected)

Total execution time: 00:00:00.017

In [49]:
EXEC SP_ADD_REVIEW_BY_TITLE 'Manas', 'Santosh', 3

: Msg 50000, Level 15, State 2, Procedure SP_ADD_REVIEW_BY_TITLE, Line 32
Multiple Matching Books

Total execution time: 00:00:00.009

In [50]:
EXEC SP_ADD_REVIEW_BY_TITLE 'Summons', 'Santosh', 3

(1 row affected)

Total execution time: 00:00:00.012

In [51]:
EXEC SP_ADD_REVIEW_BY_TITLE 'Summons2', 'Santosh', 3

: Msg 50000, Level 15, State 1, Procedure SP_ADD_REVIEW_BY_TITLE, Line 26
No Matching Book

Total execution time: 00:00:00

In [52]:
SELECT TITLE, REVIEWER, RATING FROM BOOKS,REVIEWS WHERE BOOKS.ID=BOOK_ID AND REVIEWER='Santosh'

(2 rows affected)

Total execution time: 00:00:00.004

TITLE,REVIEWER,RATING
The Accursed God,Santosh,4
Summons,Santosh,3


# Functions

- SQL SERVER ALSO ALLOWS US TO CREATE FUNCTIONS THAT ARE STORED IN DATABASE SERVER
- UNLIKE SP
    - FUNCTIONS CAN ONLY USE SELECT CALLS
    - IT CAN BE USED WITH SQL STATEMENTS.
- EXAMPLE OF FUNCTIONS
    - AVG()
    - COUNT()
    - ROUND()

## INT AVERAGE

In [59]:
CREATE FUNCTION GetFloat(
    @VALUE INT
)
RETURNS FLOAT
AS
BEGIN
    return CAST(@VALUE AS FLOAT)
END

Commands completed successfully.

Total execution time: 00:00:00.011

In [61]:
SELECT dbo.GetFloat(21)

(1 row affected)

Total execution time: 00:00:00.013

(No column name)
21


In [64]:
SELECT 
    AVG(RATING) R1,
    AVG(DBO.GETFLOAT(RATING)) R2
FROM
    REVIEWS

(1 row affected)

Total execution time: 00:00:00.002

R1,R2
4,4.363636363636363


In [69]:
CREATE FUNCTION GetReviewsByReviewer(
    @reviewer varchar(100)
)
RETURNS @result TABLE(
    BOOK_ID INT,
    REVIEW_ID INT,
    TITLE VARCHAR(100),
    REVIEWER VARCHAR(100),
    RATING INT

) -- creates a temp result table
AS
BEGIN
    
    INSERT INTO @result(BOOK_ID, REVIEW_ID, TITLE, REVIEWER, RATING)
    -- INSERT FROM SELECT
    SELECT 
        BOOKS.ID AS BOOK_ID,
        REVIEWS.ID AS REVIEW_ID,
        TITLE,
        REVIEWER,
        RATING
    FROM 
        REVIEWS
    JOIN
        BOOKS
    ON
        BOOKS.ID=BOOK_ID
    WHERE
        REVIEWER=@REVIEWER;

    return
END

Commands completed successfully.

Total execution time: 00:00:00.021

In [73]:
SELECT
    *
FROM DBO.GetReviewsByReviewer('Sanjay')

Where rating!=5;

(2 rows affected)

Total execution time: 00:00:00.005

BOOK_ID,REVIEW_ID,TITLE,REVIEWER,RATING
2,7,Manas,Sanjay,4
8,18,Rashmirathi,Sanjay,4
